In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
from tensorflow import keras 
import tensorflow 
import torch 

2023-12-04 09:51:44.876787: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-04 09:51:44.952931: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-04 09:51:45.181683: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-04 09:51:45.182659: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-04 09:51:46.204541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
torch.__version__

'2.0.0+cu117'

# Transformer Data Setup 

Defining methods to set up data to return sequences of varying lengths (n_steps) for x and y:


We also plan to try to clean up the data more exhaustively.

In [3]:
TransformerData = pd.read_csv("/home/jik19004/FilesToRun/DINGRESEARCH/DingResearchDec28th/DingLSTMFolder/FINALIZED_DATA.csv")
display(TransformerData)

,Demand,Annual Usage (kWh),Year,Month,Day,Hour,WeekDay or Weekend,HVN_lon,HVN_lat,HVN_tmpf,...,DXR_drct,DXR_sknt,DXR_vsby,MMK_lon,MMK_lat,MMK_tmpf,MMK_drct,MMK_sknt,MMK_vsby,HotTemperatures
0,3053.00,13244.454545,2011,1,1,0,0,-72.8868,41.2638,33.980000,...,0.000000,0.000000,5.000000,-72.83,41.51,28.94,0.0,0.0,4.0,0
1,2892.00,13244.454545,2011,1,1,1,0,-72.8868,41.2638,37.940000,...,0.000000,0.000000,5.000000,-72.83,41.51,28.94,0.0,0.0,4.0,0
2,2774.00,13244.454545,2011,1,1,2,0,-72.8868,41.2638,37.040000,...,0.000000,0.000000,4.000000,-72.83,41.51,33.08,150.0,5.0,5.0,0
3,2710.00,13244.454545,2011,1,1,3,0,-72.8868,41.2638,33.980000,...,0.000000,0.000000,4.000000,-72.83,41.51,33.08,150.0,4.0,6.0,0
4,2698.00,13244.454545,2011,1,1,4,0,-72.8868,41.2638,30.920000,...,0.000000,0.000000,4.000000,-72.83,41.51,28.94,100.0,3.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96422,3310.60,14064.500000,2021,12,31,19,1,-72.8868,41.2638,46.133333,...,46.666667,1.333333,6.666667,-72.83,41.51,43.00,0.0,0.0,4.0,0
96423,3148.00,14064.500000,2021,12,31,20,1,-72.8868,41.2638,46.133333,...,46.666667,1.333333,6.666667,-72.83,41.51,43.00,0.0,0.0,4.0,0
96424,2988.93,14064.500000,2021,12,31,21,1,-72.8868,41.2638,46.133333,...,46.666667,1.333333,6.666667,-72.83,41.51,43.00,0.0,0.0,4.0,0
96425,2793.70,14064.500000,2021,12,31,22,1,-72.8868,41.2638,46.133333,...,46.666667,1.333333,6.666667,-72.83,41.51,43.00,0.0,0.0,4.0,0


In [4]:
DemandData = TransformerData.loc[:, "Demand"].copy()
TransformerData.drop("Demand", axis = 1,inplace = True)
display(TransformerData)

,Annual Usage (kWh),Year,Month,Day,Hour,WeekDay or Weekend,HVN_lon,HVN_lat,HVN_tmpf,HVN_drct,...,DXR_drct,DXR_sknt,DXR_vsby,MMK_lon,MMK_lat,MMK_tmpf,MMK_drct,MMK_sknt,MMK_vsby,HotTemperatures
0,13244.454545,2011,1,1,0,0,-72.8868,41.2638,33.980000,190.000000,...,0.000000,0.000000,5.000000,-72.83,41.51,28.94,0.0,0.0,4.0,0
1,13244.454545,2011,1,1,1,0,-72.8868,41.2638,37.940000,63.333333,...,0.000000,0.000000,5.000000,-72.83,41.51,28.94,0.0,0.0,4.0,0
2,13244.454545,2011,1,1,2,0,-72.8868,41.2638,37.040000,200.000000,...,0.000000,0.000000,4.000000,-72.83,41.51,33.08,150.0,5.0,5.0,0
3,13244.454545,2011,1,1,3,0,-72.8868,41.2638,33.980000,130.000000,...,0.000000,0.000000,4.000000,-72.83,41.51,33.08,150.0,4.0,6.0,0
4,13244.454545,2011,1,1,4,0,-72.8868,41.2638,30.920000,130.000000,...,0.000000,0.000000,4.000000,-72.83,41.51,28.94,100.0,3.0,5.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96422,14064.500000,2021,12,31,19,1,-72.8868,41.2638,46.133333,86.666667,...,46.666667,1.333333,6.666667,-72.83,41.51,43.00,0.0,0.0,4.0,0
96423,14064.500000,2021,12,31,20,1,-72.8868,41.2638,46.133333,86.666667,...,46.666667,1.333333,6.666667,-72.83,41.51,43.00,0.0,0.0,4.0,0
96424,14064.500000,2021,12,31,21,1,-72.8868,41.2638,46.133333,86.666667,...,46.666667,1.333333,6.666667,-72.83,41.51,43.00,0.0,0.0,4.0,0
96425,14064.500000,2021,12,31,22,1,-72.8868,41.2638,46.133333,86.666667,...,46.666667,1.333333,6.666667,-72.83,41.51,43.00,0.0,0.0,4.0,0


# Scaling Our Data so that one particular feature wouldn't dominate the loss in particular! 

# Splitting the Data into Training, Validation, and Testing! 

In [5]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameter 
import tensorflow as tf 
from tensorflow import keras as keras 

In [6]:
def return_sequences(data, outputData, input_n_steps, output_n_steps):
    X = []
    Y = []
    length = len(data)
    for i in range(0,length, output_n_steps):
        input_indx = i + input_n_steps
        output_indx = input_indx + output_n_steps 
        if (output_indx > len(data)): # we need to have equally split sequences. 
            break               # The remaining data that cannot fit into a fixed
                                # sequence will immediately be cut!
        else:
            Xsample = data.iloc[i:input_indx, :] # get the previous data
            Ysample = outputData[input_indx:output_indx]
            X.append(Xsample)
            Y.append(Ysample) 
    X = np.asarray(X).astype('float64')
    Y = np.asarray(Y).astype('float64')
    return (X, Y)


In [7]:
from sklearn.preprocessing import StandardScaler 
def scaleTheData(data):
    scaler = StandardScaler()
    # split the data first. 
    data2 = scaler.fit_transform(data)
    data = pd.DataFrame(data2, columns = data.columns)
    return data

def splitDataAndScale(data, output):
    
    TrainingData = scaleTheData(data.iloc[:61363, :].copy()) # The index for Janruary 1st 2018 basically, don't necessarily hard code it. 
    TrainingOutput = output[:61363].copy()

    RemainingData = data.iloc[61363:, :].copy()
    RemainingOutput = output[61363:].copy()

    ValidationData = scaleTheData(RemainingData.iloc[:int(0.5*len(RemainingData)), :].copy())
    ValidationOutput = RemainingOutput[:int(0.5*len(RemainingOutput))].copy()

    TestingData = scaleTheData(RemainingData.iloc[int(0.5 * len(RemainingData)):, :].copy())
    TestingOutput = RemainingOutput[int(0.5 * len(RemainingData)):].copy()

    TrainingSequences = return_sequences(TrainingData, TrainingOutput, 18, 12)

    TransformedTrainingData = TrainingSequences[0]
    TransformedTrainingOutput = TrainingSequences[1]

    ValidationSequences = return_sequences(ValidationData, ValidationOutput, 18,12)

    TransformedValidationData = ValidationSequences[0]
    TransformedValidationOutput = ValidationSequences[1]

    TestingSequences = return_sequences(TestingData, TestingOutput, 18, 12)

    TransformedTestingData = TestingSequences[0]
    TransformedTestingOutput = TestingSequences[1]



    return (TransformedTrainingData, TransformedTrainingOutput, TransformedValidationData, TransformedValidationOutput, 
    TransformedTestingData, TransformedTestingOutput)

In [8]:
data = splitDataAndScale(TransformerData, DemandData)

In [9]:
print(data[0].shape)

(5112, 18, 55)


In [10]:
print(data[2].shape)

(1459, 18, 55)


In [11]:
print(data[4].shape)

(1459, 18, 55)


In [12]:
trainingData = data[0]
trainingOutput = data[1]
validationData = data[2]
validationOutput = data[3]
testingData = data[4]
testingnOutput = data[5]

In [13]:
print(len(data[2]))

1459


In [14]:
TrainingData = [];  
for i in range(5112): 
    TrainingData.append((data[0][i], data[1][i]))

In [15]:
print(validationData.shape)
print(validationOutput.shape)

ValidationData = []; 
for i in range(1459):
    ValidationData.append((data[2][i], data[3][i]))


(1459, 18, 55)
(1459, 12)


# Transformer Model Construction


In [16]:
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [17]:
def getPositionalEncoding1(batch_size, seq_len, d, n = 10000):
        P = np.zeros((batch_size,seq_len, d))
        for z in range(batch_size):
            for k in range(seq_len):
                for i in np.arange(int(d/2)):
                    denominator = np.power(n, 2*i/d)
                    P[z, k, 2*i] = np.sin(k/denominator)
                    P[z,k, 2*i+1] = np.cos(k/denominator)
        P = torch.tensor(P).to()
        return torch.tensor(P)

print(type(getPositionalEncoding1(10, 10, 10)))


<class 'torch.Tensor'>


/tmp/ipykernel_26880/1688313836.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(P)


In [18]:
class TimeSeriesTransformer(nn.Module): 
    def __init__(self, num_layers, dropout = 0.4, outfeatures1 = 160, outfeatures2 = 160, dim_feedforward = 2048, output_num = 6):
        super().__init__()

        self.Linear0 = nn.Linear(in_features = 55, out_features = 110)
        self.encoder = nn.TransformerEncoder(nn.TransformerEncoderLayer(d_model = 110, nhead = 5, dim_feedforward = dim_feedforward, dropout = dropout, batch_first = True), num_layers = num_layers)
        self.batchnorm = nn.BatchNorm1d(num_features = 18)      
        self.Flatten = nn.Flatten() # take into account that we're doing this in batches!! 
        
        self.Linear1 = nn.Linear(in_features = 110, out_features = outfeatures1);
        self.activation1 = nn.LeakyReLU(negative_slope = 0.25, inplace = True)
        self.batchnorm1 = nn.BatchNorm1d(num_features = 18)
        self.dropoutLayer1 = nn.Dropout(dropout)


        # We then slice the last half! 
        self.Linear2 = nn.Linear(in_features = outfeatures1, out_features = outfeatures2); 
        self.activation2 = nn.LeakyReLU(negative_slope = 0.25, inplace = True)
        self.batchnorm2 = nn.BatchNorm1d(num_features = 18)
        self.dropoutLayer2 = nn.Dropout(dropout);

        self.Linear3 = nn.Linear(in_features= outfeatures2, out_features = 120)
        self.activation3 = nn.LeakyReLU(negative_slope = 0.25, inplace = True)
        self.batchnorm3 = nn.BatchNorm1d(num_features = 18)
        self.dropoutLayer3 = nn.Dropout(dropout);


        self.Linear4 = nn.Linear(in_features= 120, out_features = 180)
        self.activation4 = nn.LeakyReLU(negative_slope = 0.25, inplace = True)
        self.batchnorm4 = nn.BatchNorm1d(num_features = 18)
        self.dropoutLayer4 = nn.Dropout(dropout);


        self.Linear5 = nn.Linear(in_features = 180 , out_features = 92) # we will then compute using 2 more fully connected layers. 
        self.activation5 = nn.ELU(inplace = True);
        self.dropoutLayer5 = nn.Dropout(dropout);
        self.batchnorm5 = nn.BatchNorm1d(num_features = 18)
        

        self.Linear6 = nn.Linear(in_features= 92, out_features = 64)
        self.activation6 = nn.ELU(inplace = False);
        self.dropoutLayer6 = nn.Dropout(dropout);
        self.batchnorm6 = nn.BatchNorm1d(num_features = 18)
        
        self.Linear7 = nn.Linear(in_features = 64, out_features= 16)
        self.activation7 = nn.ReLU(inplace= False)


        
        #self.Linear4 = nn.Linear(in_features = outfeatures3, out_features = 64)
        #self.activation4 = nn.LeakyReLU(negative_slope = 0.25, inplace = False)
        #self.dropoutLayer4 = nn.Dropout(dropout);


        self.Linear8 = nn.Linear(in_features = 16 * 18, out_features = output_num)

    def getPositionalEncoding(self, batch_size = 128, seq_len = 18, d = 110, n = 10000, device = torch.device("cuda")):
        P = np.zeros((batch_size,seq_len, d))
        for z in range(batch_size):
            for k in range(seq_len):
                for i in np.arange(int(d/2)):
                    denominator = np.power(n, 2*i/d)
                    P[z, k, 2*i] = np.sin(k/denominator)
                    P[z,k, 2*i+1] = np.cos(k/denominator)
        P = torch.tensor(P)
        P = P.type("torch.FloatTensor")
        return P.to(device)

    def forward(self, src):
        
        val = self.Linear0(src)
        val = val + self.getPositionalEncoding(batch_size = val.size(0), seq_len = val.size(1), d = val.size(2))
        val = self.encoder(val) # this would be (72, 50)
        val = self.batchnorm(val)


        val = self.Linear1(val) # this would then become (72, 6) 
        val = self.activation1(val) 
        val = self.batchnorm1(val) 
        val = self.dropoutLayer1(val) 


        val = self.Linear2(val) 
        val = self.activation2(val)
        val = self.batchnorm2(val)
        val = self.dropoutLayer2(val)

        val = self.Linear3(val)
        val = self.activation3(val)
        val = self.batchnorm3(val)
        val = self.dropoutLayer3(val)
                                    # Somehow, we need to conver the tensor of a size [28, 1, 55] into a 2 dimensional vector 
                                    # where it preserves dimension of the zeroeth index. But have it up to 2nd dimension. 
                                    # [28, etc] use tensor = torch.randn(x,y,1).squeeze(-1) where x = 10, y = 20, z = 30
        val = self.Linear4(val) 
        val = self.activation4(val)
        val = self.batchnorm4(val)
        val = self.dropoutLayer4(val) 
         # convert to 2 dimensional tensor 

        val = self.Linear5(val)
        val = self.activation5(val)
        val = self.batchnorm5(val)
        val = self.dropoutLayer5(val)
        
        val = self.Linear6(val)
        val = self.activation6(val)
        val = self.batchnorm6(val)
        val = self.dropoutLayer6(val)
        
        
        
        val = self.Linear7(val)
        val = self.activation7(val)        
        val = val.view(-1, val.size(1) * val.size(2))   


        val = self.Linear8(val) # This returns an output of a varying number. 
        val = val.squeeze()
        return val

In [19]:
import optuna 
class TimeSeriesDataset(Dataset):
    def __init__(self, data, output):
        data = torch.tensor(data).float(); 
        output = torch.tensor(output).float() 

        self.data = data 
        self.output = output; 

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx];
        y = self.output[idx];
        return x, y;

# use the past 72 hours in advance and then predict the 1st hour, 6th hour, 12 hours! 

def evaluate(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for data, target in val_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            loss = criterion(output, target)
            running_loss += loss.item() * target.size(0)
    return running_loss / len(val_loader.dataset)

def Train_and_Evaluate(train_loader, val_loader, device, params, numEpochs, early_stop_epochs):
    #num_layers, dropout = 0.1, outfeatures1 = 16, outfeatures2 = 16, outfeatures3 = 16, outfeatures4 = 16, dim_feedforward = 2048, output_num = 6
    model = TimeSeriesTransformer(num_layers = params[0], outfeatures1 = params[1], outfeatures2 = params[2],
                                dim_feedforward = params[3], output_num = 12)
    model = model.to(device);
    LossFunction = torch.nn.L1Loss();
    best_val_loss = float('inf')
    early_stop_count = 0
    print("num layers:", params[0])
    print("outfeatures1:", params[1])
    print("outfeatures2:", params[2])
    print("dim_feedforward:", params[3])
    
    Optimizer = torch.optim.Adagrad(params = model.parameters())
    for epoch in range(0,numEpochs): 
        Training_Loss = 0; 
        total_samples = 0; 
        for input, output in train_loader:
            input = input.to(device); 
            output = torch.squeeze(output, 1); 
            output = output.to(device); 
            predictedVal = model(input)
            #predictedVal = torch.squeeze(predictedVal, 1)
            Optimizer.zero_grad();
            batchLoss = LossFunction(predictedVal, output);
            batchLoss.backward(); 
            Optimizer.step();
            Training_Loss += batchLoss * output.size(0) #* output.size(0);
            total_samples += output.size(0)
        Training_Loss = Training_Loss/total_samples


        Validation_Loss = 0; 
        print("passed ", epoch, "epoch", "Training Loss: ", Training_Loss," ", end = "")
        with torch.no_grad():
            total_val_samples = 0; 
            Validation_Loss = 0; 
            for val_input, val_output in val_loader:
                val_input = val_input.to(device); 
                val_output = torch.squeeze(val_output,1);
                val_output = val_output.to(device);

                predictedVal = model(val_input)
                Validation_Loss += LossFunction(val_output, predictedVal) * val_output.size(0)
                total_val_samples += val_output.size(0)
            Validation_Loss = Validation_Loss/total_val_samples
            print("Validation Loss: ", Validation_Loss)

            if Validation_Loss < best_val_loss:
                best_val_loss = Validation_Loss
                early_stop_count = 0;
                torch.save(model, "//home//jik19004//FilesToRun//DINGRESEARCH//DingResearchDec28th//DingTransformerFolderTransformer12"); #We go save with the transformer!!!
            else:
                early_stop_count +=1 
            if early_stop_count >= early_stop_epochs:
                return best_val_loss; 
            
            

    return best_val_loss; 

def predict(model, data_loader, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for data, _ in data_loader:
            data = data.to(device)
            output = model(data)
            predictions.append(output.cpu().numpy())
    return np.concatenate(predictions)

Validation_Loss = 0; 
        print("passed ", epoch, "epoch", "Training Loss: ", Training_Loss," ", end = "")
        with torch.no_grad():
            count1 = 0;
            for val_input, val_output in val_loader:
                val_input = val_input.to(device); 
                val_output = torch.squeeze(val_output,1);
                val_output = val_output.to(device);

                predictedVal = model(val_input);
                predictedVal = torch.squeeze(predictedVal, 1)
                batchLoss = LossFunction(val_output, predictedVal);
                Validation_Loss += batchLoss # * val_output.size(0);
                count1 += 
            Validation_Loss = Validation_Loss/count1
            print("Validation Loss: ", Validation_Loss)

            if Validation_Loss < best_val_loss:
                best_val_loss = Validation_Loss
                early_stop_count = 0;
            else:
                early_stop_count +=1 
            if early_stop_count >= early_stop_epochs:
                return best_val_loss; 

# GO DEAL WITH NAN VALUES IN OUR DATA!!!!

In [20]:
TrainingData = TimeSeriesDataset(data[0],data[1])
ValidationData = TimeSeriesDataset(data[2],data[3])
TestingData = TimeSeriesDataset(data[4],data[5])

In [21]:
TrainingLoader = DataLoader(TrainingData, batch_size = 92);
ValidationLoader = DataLoader(ValidationData, batch_size = 128);
TestingLoader = DataLoader(TestingData, batch_size = 128);

In [22]:
def objective(trial):
    #num_layers = params[0], outfeatures = params[1], dim_feedforward = params[2]
    params = [trial.suggest_int("num_layers", low = 2, high = 3, step = 2), 
              trial.suggest_int("out_features1", low = 224, high = 288, step = 32),
              trial.suggest_int("out_features2", low = 224, high = 288, step = 32),
              trial.suggest_int("dim_feedforward", low = 176, high = 208, step = 8)];
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu");
    return Train_and_Evaluate(TrainingLoader, ValidationLoader ,device, params, 260, 10);  

In [23]:
import joblib 
study_name = 'sqlite:///example12.db'
study = optuna.create_study(direction="minimize", sampler=optuna.samplers.TPESampler(), study_name= "Transformer12", load_if_exists= True, storage='sqlite:///example12.db')
joblib.dump(study, "study12.pkl") 
study.optimize(objective, n_trials = 120) 

[I 2023-12-04 09:51:53,129] Using an existing study with name 'Transformer12' instead of creating a new one.
/home/jik19004/anaconda3/lib/python3.9/site-packages/optuna/distributions.py:716: UserWarning: The distribution is specified by [2, 3] and step=2, but the range is not divisible by `step`. It will be replaced by [2, 2].
  warnings.warn(


num layers: 2
outfeatures1: 256
outfeatures2: 288
dim_feedforward: 176


[W 2023-12-04 09:51:57,766] Trial 103 failed with parameters: {'num_layers': 2, 'out_features1': 256, 'out_features2': 288, 'dim_feedforward': 176} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/jik19004/anaconda3/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_26880/563636019.py", line 8, in objective
    return Train_and_Evaluate(TrainingLoader, ValidationLoader ,device, params, 260, 10);
  File "/tmp/ipykernel_26880/3275244228.py", line 53, in Train_and_Evaluate
    predictedVal = model(input)
  File "/home/jik19004/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  File "/tmp/ipykernel_26880/1758547116.py", line 71, in forward
    val = self.Linear0(src)
  File "/home/jik19004/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1501, in _

KeyboardInterrupt: 

In [24]:
print("Parameters:", study.best_params)
print("MAE for validation set:", study.best_value)

Parameters: {'dim_feedforward': 184, 'num_layers': 2, 'out_features1': 256, 'out_features2': 288}
MAE for validation set: 428.1627502441406


In [ ]:
predict()